# MOVIES PROJECT

necesary imports

In [123]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

framing function

In [124]:
def frame_it(list_of_pd_series):
    return pd.concat(list_of_pd_series, axis=1)

data urls

In [125]:
URL_1_50     = "https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,asc"
URL_51_100   = "https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,asc&start=51&ref_=adv_nxt"

responses

In [126]:
response_1_50 = requests.get(URL_1_50)
response_51_100 = requests.get(URL_51_100)

soupify the responses

In [127]:
soup_1_50 = BeautifulSoup(response_1_50.text,'html.parser')
soup_51_100 = BeautifulSoup(response_51_100.text,'html.parser')

## RELEASE DATES

traverse the soup in search for the dates

In [128]:
movie_release_dates_1_50 = soup_1_50.findAll('span',class_ = 'lister-item-year text-muted unbold')
movie_release_dates_51_100 = soup_51_100.findAll('span',class_ = 'lister-item-year text-muted unbold')

movie_release_dates = movie_release_dates_1_50 + movie_release_dates_51_100

date cleaning function

In [129]:
def clean_date(date_string):

    # fix stupid strings
    date_string = (date_string.text).replace("(", "")
    date_string = date_string.replace(")", "")
    date_string = date_string.replace("I","")
    date_string = date_string.strip()

    # change them into integers
    date_int = int(date_string)

    return date_int

create an empty list for release dates

In [130]:
# list_rel_dates_1_50 = []
# list_rel_dates51_100 = []

list_rel_dates = []

process dates

In [131]:
for str_date in movie_release_dates:
    list_rel_dates.append( clean_date(str_date) )

make them into pd.Series

In [132]:
rel_dates_series = pd.Series(list_rel_dates, name="Release Date")

## RUNTIMES

traverse the soup in search for runtimes

In [133]:
runtimes_1_50 = soup_1_50.findAll('span',class_ = "runtime")
runtimes_51_100 = soup_51_100.findAll('span',class_ = "runtime")

runtimes = runtimes_1_50 + runtimes_51_100

create empty lists for runtimes

In [134]:
movie_runtimes = []

create a time-cleaning function

In [135]:
def clean_time(time_string):

    time_string = time_string.split()
    time_string = time_string[0]

    return int(time_string)

process runtimes

In [136]:
for runtime in runtimes:
    movie_runtimes.append( clean_time(runtime.text) )

make them into pd.Series

In [137]:
runtimes_series = pd.Series(movie_runtimes, name="Runtime")

## MOVIE NAMES

traverse the soup

In [138]:
movie_names_1_50 = soup_1_50.find_all(name="h3", class_="lister-item-header")
movie_names_51_100 = soup_51_100.find_all(name="h3", class_="lister-item-header")

movie_names = movie_names_1_50 + movie_names_51_100

create an empty list for movie names

In [139]:
names_list = []
len(movie_names)

100

In [140]:
for idx, mn in enumerate( movie_names ):
    print(f"-------------{idx}-------------", mn.text)

-------------0------------- 
1.
M - morderca
(1931)

-------------1------------- 
2.
Obywatel Kane
(1941)

-------------2------------- 
3.
Deszczowa piosenka
(1952)

-------------3------------- 
4.
Zawrót głowy
(1958)

-------------4------------- 
5.
Północ - północny zachód
(1959)

-------------5------------- 
6.
2001: Odyseja kosmiczna
(1968)

-------------6------------- 
7.
Okręt
(1981)

-------------7------------- 
8.
Gwiezdne wojny: Część VI - Powrót Jedi
(1983)

-------------8------------- 
9.
Amadeusz
(1984)

-------------9------------- 
10.
Obcy - decydujące starcie
(1986)

-------------10------------- 
11.
Idź i patrz
(1985)

-------------11------------- 
12.
Wściekłe psy
(1992)

-------------12------------- 
13.
Braveheart - Waleczne Serce
(1995)

-------------13------------- 
14.
Toy Story
(1995)

-------------14------------- 
15.
Buntownik z wyboru
(1997)

-------------15------------- 
16.
American Beauty
(1999)

-------------16------------- 
17.
Requiem dla snu
(2000)

---

process names

In [141]:
for mn in movie_names:
    names_list.append(list(mn)[3].text)

make them into pd.Series

In [142]:
names_series = pd.Series(names_list, name="Movie name")

## MOVIE TYPES

traverse the soup

In [149]:
genres_1_50 = soup_1_50.find_all(name="span", class_="genre")
genres_51_100 = soup_51_100.find_all(name="span", class_="genre")

genres = genres_1_50 + genres_51_100

create an empty list, process and serialise it

In [153]:
genres_list = []
for gen in genres:
    string = list(gen)[0].replace("\n", "").strip()
    genres_list.append(string)
genres_series = pd.Series(genres_list, name="Genre")

0         Crime, Mystery, Thriller
1                   Drama, Mystery
2         Comedy, Musical, Romance
3       Mystery, Romance, Thriller
4     Adventure, Mystery, Thriller
                  ...             
95                    Crime, Drama
96                    Crime, Drama
97            Action, Crime, Drama
98                    Crime, Drama
99                           Drama
Name: Genre, Length: 100, dtype: object

## PROFIT!

In [155]:
all_series = [names_series, rel_dates_series, runtimes_series, genres_series]
# df = frame_it(all_series)


# %timeit df = pd.DataFrame([all_series, runtimes_series]).T # this is basicaly shit.
df = pd.concat(all_series, axis=1)

action_df = df.loc[df['Genre'].str.contains('Action')]
action_df

,Movie name,Release Date,Runtime,Genre
7,Gwiezdne wojny: Część VI - Powrót Jedi,1983,131,"Action, Adventure, Fantasy"
9,Obcy - decydujące starcie,1986,137,"Action, Adventure, Sci-Fi"
29,Poszukiwacze zaginionej arki,1981,115,"Action, Adventure"
31,Księżniczka Mononoke,1997,134,"Animation, Action, Adventure"
33,Oldboy,2003,120,"Action, Drama, Mystery"
38,Mroczny Rycerz powstaje,2012,164,"Action, Adventure"
41,Avengers: Wojna bez granic,2018,149,"Action, Adventure, Sci-Fi"
42,Avengers: Koniec gry,2019,181,"Action, Adventure, Drama"
43,Spider-Man Uniwersum,2018,117,"Animation, Action, Adventure"
56,Terminator 2: Dzień sądu,1991,137,"Action, Sci-Fi"
